In [ ]:
!pip install tensorflow
!pip install keras


  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0


In [ ]:
import json
import pandas as pd
import numpy as np
import pika
from keras.models import Sequential
from keras.layers import LSTM, Dense



import pickle

EXCHANGE_NAME = 'prediction_exchangesa'
QUEUE_NAME = 'lstm-result-queue'
MODEL_FILE_PATH = 'lstm_model.pkl'

def train_and_save_lstm(df_train):
    # Normalize the data
    normalized_data = (df_train - df_train.min()) / (df_train.max() - df_train.min())

    # Convert the DataFrame to a numpy array
    training_data = normalized_data.values

    # Split the data into input and output
    x_train, y_train = [], []
    for i in range(len(training_data) - 10):
        x_train.append(training_data[i:i+9])  # Use the last 9 values to predict the next one
        y_train.append(training_data[i+10])  # Predict the 10th value

    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshape the data for LSTM input
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', input_shape=(x_train.shape[1], 1)))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(x_train, y_train, epochs=50, batch_size=1)

    # Save the trained model to a file
    model.save(MODEL_FILE_PATH)

def load_and_predict_lstm(df_forecast):
    # Load the trained model from the file
    model = Sequential()
    model = model.load(MODEL_FILE_PATH)

    # Normalize the forecast data
    normalized_forecast = (df_forecast - df_forecast.min()) / (df_forecast.max() - df_forecast.min())

    # Convert the DataFrame to a numpy array
    forecast_data = normalized_forecast.values

    # Prepare data for forecasting
    x_forecast = forecast_data[-9:]  # Use the last 9 values for forecasting
    x_forecast = np.reshape(x_forecast, (1, x_forecast.shape[0], 1))

    # Make predictions
    predicted_value = model.predict(x_forecast)

    # Denormalize the predicted value
    predicted_value = predicted_value * (df_forecast.max() - df_forecast.min()) + df_forecast.min()

    return predicted_value.flatten()

def callback(ch, method, properties, body):
    message = body.decode('latin-1')
    print(f"Received: {message}")

    try:
        json_object = json.loads(message)
        start_date = json_object['startDate']
        csv_path = json_object['temperatureData']

        # Read only the necessary columns
        df1 = pd.read_csv("lettuce_dataset.csv", usecols=['Date', 'Temperature'], encoding='ISO-8859-1')

        # Convert 'Date' to datetime
        df1['Date'] = pd.to_datetime(df1['Date'], errors='coerce')

        # Drop rows with missing dates
        df1 = df1.dropna(subset=['Date'])

        # Set 'Date' as the index
        df1.set_index('Date', inplace=True)

        # Train and save the LSTM model
        train_and_save_lstm(df1)

        # Convert the 'date' column to datetime format
        df_forecast = pd.DataFrame(csv_path)
        df_forecast['date'] = pd.to_datetime(df_forecast['date'], format='%Y-%m-%d')

        # Set 'date' as the index
        df_forecast.set_index('date', inplace=True)  # Corrected line using 'date' column

        # Load the trained model and make predictions
        forecast_values = load_and_predict_lstm(df_forecast)

        prediction_dict = [{'date': str(key), 'temperature': value} for key, value in zip(df_forecast.index, forecast_values)]

        connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
        channel = connection.channel()
        channel.exchange_declare(exchange=EXCHANGE_NAME, exchange_type='fanout', durable=True)
        channel.queue_declare(queue=QUEUE_NAME, durable=True)
        channel.queue_bind(exchange=EXCHANGE_NAME, queue=QUEUE_NAME)

        print("---------")

        prediction_message = json.dumps({"prediction": prediction_dict})
        print(f"Number of elements in prediction_dict: {prediction_dict}")

        channel.basic_publish(exchange=EXCHANGE_NAME, routing_key='', body=prediction_message)
        connection.close()

    except KeyError as e:
        print(f"Error processing message: {e}")

if __name__ == '__main__':
    connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
    channel = connection.channel()
    channel.exchange_declare(exchange=EXCHANGE_NAME, exchange_type='fanout', durable=True)
    channel.queue_declare(queue=QUEUE_NAME, durable=True)
    channel.queue_bind(exchange=EXCHANGE_NAME, queue=QUEUE_NAME)
    channel.basic_consume(queue=QUEUE_NAME, on_message_callback=callback, auto_ack=True)
    print('Waiting for messages. To exit press CTRL+C')
    channel.start_consuming()
